# { Module 10 Homework }

1. Import the "babies.xlsx" dataset. See below for information on the columns:

    * bwt - birth weight of newborn baby
    * gestation	- gestation length (weeks)
    * parity - previously pregnant (0 = no; 1 = yes)
    * age - age of mother
    * height - height of mother (inches)	
    * weight - weight of mother (pounds)
    * smoke - smoking status of mother (0 = nonsmoker; 1 = smoker)

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats

babies = pd.read_excel("babies.xlsx")

babies.head()

,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,0,27.0,62.0,100.0,0.0
1,113,282.0,0,33.0,64.0,135.0,0.0
2,128,279.0,0,28.0,64.0,77.0,1.0
3,123,NaN,0,36.0,69.0,190.0,0.0
4,108,282.0,0,23.0,67.0,125.0,1.0


2. Preview the first few rows of the dataset. Complete the following checks for your dataset:

    * output the summary information for the dataset, what types of variables make up this dataset?
    * is there any missing data in this dataset?

In [5]:
babies.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   bwt        1236 non-null   int64  
 1   gestation  1223 non-null   float64
 2   parity     1236 non-null   int64  
 3   age        1234 non-null   float64
 4   height     1214 non-null   float64
 5   weight     1200 non-null   float64
 6   smoke      1226 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 67.7 KB


3. Handle the missing data in the dataset -- there isn't much, so we can drop all the rows that have at least one missing value. How many rows were dropped from the dataset?

In [6]:
babies.isnull().sum()

bwt           0
gestation    13
parity        0
age           2
height       22
weight       36
smoke        10
dtype: int64

In [7]:
babies.dropna(inplace = True)
babies.head()

,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,0,27.0,62.0,100.0,0.0
1,113,282.0,0,33.0,64.0,135.0,0.0
2,128,279.0,0,28.0,64.0,77.0,1.0
4,108,282.0,0,23.0,67.0,125.0,1.0
5,136,286.0,0,25.0,62.0,93.0,0.0


4. There are two qualitative variables in this dataset. What are they? How do you know they are qualitative variables?

5. Replace the values in the qualitative variables with meaningful labels that describe the different groups. Use any method you've learned in previous modules to complete this task. 

In [8]:
babies["smoke"].replace([0, 1], ["N", "Y"], inplace = True)
babies["parity"].replace([0, 1], ["N", "Y"], inplace = True)

babies

,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
2,128,279.0,N,28.0,64.0,77.0,Y
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
...,...,...,...,...,...,...,...
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1233,130,291.0,N,30.0,65.0,67.0,Y
1234,125,281.0,Y,21.0,65.0,110.0,N


6. Before we move forward with any statistics, let's identify and remove any outliers from the dataset. Using the IQR method, search for outliers in the 5 numeric variables. This will take some time and organization, be careful with your code! Make sure you keep track of how many rows/outliers are removed. 

In [9]:
q1 = babies["bwt"].quantile(.25)
q3 = babies["bwt"].quantile(.75)

print("Q1:", q1)
print("Q3:", q3)

Q1: 108.0
Q3: 131.0


In [10]:
iqr = q3 - q1
print(iqr)

23.0


In [11]:
top = q3 + (iqr * 1.5)
bottom = q1 - (iqr * 1.5)


print("Upper Limit:", top)
print("Lower Limit:", bottom)

Upper Limit: 165.5
Lower Limit: 73.5


In [12]:
iqr_outliers = babies.loc[(babies['bwt'] > top) | (babies['bwt'] < bottom)].index

print("INDEX VALUES:", iqr_outliers)

INDEX VALUES: Int64Index([ 239,  309,  361,  462,  500,  529,  556,  594,  632,  709,  738,
             747,  829,  904,  912,  978, 1021, 1035, 1063, 1065, 1099, 1139,
            1148, 1169],
           dtype='int64')


In [13]:
babies.loc[[ 239,  309,  361,  462,  500,  529,  556,  594,  632,  709,  738,
             747,  829,  904,  912,  978, 1021, 1035, 1063, 1065, 1099, 1139,
            1148, 1169]]

,bwt,gestation,parity,age,height,weight,smoke
239,173,293.0,N,30.0,63.0,110.0,N
309,71,281.0,N,32.0,60.0,117.0,Y
361,71,234.0,N,32.0,64.0,110.0,Y
462,68,223.0,N,32.0,66.0,149.0,Y
500,69,232.0,N,31.0,59.0,103.0,Y
529,71,277.0,N,40.0,69.0,135.0,N
556,174,281.0,N,37.0,67.0,155.0,N
594,170,303.0,Y,21.0,64.0,129.0,N
632,176,293.0,Y,19.0,68.0,180.0,N
709,166,299.0,N,26.0,68.0,140.0,N


In [14]:
babies = babies.drop(iqr_outliers)
babies

,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
2,128,279.0,N,28.0,64.0,77.0,Y
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
...,...,...,...,...,...,...,...
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1233,130,291.0,N,30.0,65.0,67.0,Y
1234,125,281.0,Y,21.0,65.0,110.0,N


In [15]:
q1 = babies["gestation"].quantile(.25)
q3 = babies["gestation"].quantile(.75)
iqr = q3 - q1
iqr

16.0

In [16]:
top = q3 + (iqr * 1.5)
bottom = q1 - (iqr * 1.5)

print("Upper Limit:", top)
print("Lower Limit:", bottom)

Upper Limit: 312.0
Lower Limit: 248.0


In [17]:
iqr_outliers = babies.loc[(babies['gestation'] > top) | (babies['gestation'] < bottom)].index

print("INDEX VALUES:", iqr_outliers)
babies = babies.drop(iqr_outliers)
babies

INDEX VALUES: Int64Index([   6,    7,   10,   59,   63,   66,  119,  129,  155,  188,  192,
             198,  210,  215,  217,  234,  240,  253,  260,  279,  345,  373,
             394,  440,  460,  484,  511,  523,  630,  685,  699,  710,  726,
             746,  761,  769,  778,  784,  828,  830,  833,  869,  927,  952,
             969, 1002, 1020, 1026, 1074, 1134, 1140, 1142, 1146, 1152, 1172,
            1178, 1199, 1206, 1207, 1217, 1219, 1226],
           dtype='int64')


,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
2,128,279.0,N,28.0,64.0,77.0,Y
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
...,...,...,...,...,...,...,...
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1233,130,291.0,N,30.0,65.0,67.0,Y
1234,125,281.0,Y,21.0,65.0,110.0,N


In [18]:
q1 = babies["age"].quantile(.25)
q3 = babies["age"].quantile(.75)
iqr = q3 - q1
top = q3 + (iqr * 1.5)
bottom = q1 - (iqr * 1.5)

print("Upper Limit:", top)
print("Lower Limit:", bottom)
iqr_outliers = babies.loc[(babies['age'] > top) | (babies['age'] < bottom)].index

print("INDEX VALUES:", iqr_outliers)
babies = babies.drop(iqr_outliers)
babies

Upper Limit: 43.0
Lower Limit: 11.0
INDEX VALUES: Int64Index([633, 1070], dtype='int64')


,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
2,128,279.0,N,28.0,64.0,77.0,Y
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
...,...,...,...,...,...,...,...
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1233,130,291.0,N,30.0,65.0,67.0,Y
1234,125,281.0,Y,21.0,65.0,110.0,N


In [19]:
q1 = babies["height"].quantile(.25)
q3 = babies["height"].quantile(.75)
iqr = q3 - q1
top = q3 + (iqr * 1.5)
bottom = q1 - (iqr * 1.5)

print("Upper Limit:", top)
print("Lower Limit:", bottom)
iqr_outliers = babies.loc[(babies['height'] > top) | (babies['height'] < bottom)].index

print("INDEX VALUES:", iqr_outliers)
babies = babies.drop(iqr_outliers)
babies

Upper Limit: 72.0
Lower Limit: 56.0
INDEX VALUES: Int64Index([434, 1208], dtype='int64')


,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
2,128,279.0,N,28.0,64.0,77.0,Y
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
...,...,...,...,...,...,...,...
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1233,130,291.0,N,30.0,65.0,67.0,Y
1234,125,281.0,Y,21.0,65.0,110.0,N


In [20]:
q1 = babies["weight"].quantile(.25)
q3 = babies["weight"].quantile(.75)
iqr = q3 - q1
top = q3 + (iqr * 1.5)
bottom = q1 - (iqr * 1.5)

print("Upper Limit:", top)
print("Lower Limit:", bottom)
iqr_outliers = babies.loc[(babies['weight'] > top) | (babies['weight'] < bottom)].index

print("INDEX VALUES:", iqr_outliers)
babies = babies.drop(iqr_outliers)
babies

Upper Limit: 171.5
Lower Limit: 79.5
INDEX VALUES: Int64Index([   2,   14,   21,   23,   27,   41,   88,  117,  125,  147,  149,
             162,  180,  181,  183,  222,  287,  310,  314,  344,  411,  426,
             458,  509,  512,  522,  528,  563,  595,  608,  622,  695,  696,
             723,  733,  849,  858,  859,  865,  888,  911,  924,  935, 1008,
            1154, 1167, 1212, 1233],
           dtype='int64')


,bwt,gestation,parity,age,height,weight,smoke
0,120,284.0,N,27.0,62.0,100.0,N
1,113,282.0,N,33.0,64.0,135.0,N
4,108,282.0,N,23.0,67.0,125.0,Y
5,136,286.0,N,25.0,62.0,93.0,N
8,120,289.0,N,25.0,62.0,125.0,N
...,...,...,...,...,...,...,...
1230,132,270.0,N,27.0,65.0,126.0,N
1231,113,275.0,Y,27.0,60.0,100.0,N
1232,128,265.0,N,24.0,67.0,120.0,N
1234,125,281.0,Y,21.0,65.0,110.0,N


7. Describe the characteristics of your qualitative variables by doing the following:

    * Determine the frequencies of each group within each categorical variable. Which groups have the highest frequency?
    * Determine the relative frequencies of each group within each categorical variable. Which groups have the highest relative frequencies?
    * Create a crosstab table using both the categorical variables in your dataset. How many mothers are smokers who have been pregnant before?

In [21]:
babies["parity"].value_counts()

N    754
Y    282
Name: parity, dtype: int64

In [22]:
babies["parity"].value_counts(normalize=True)

N    0.727799
Y    0.272201
Name: parity, dtype: float64

In [23]:
babies["smoke"].value_counts()

N    631
Y    405
Name: smoke, dtype: int64

In [24]:
babies["smoke"].value_counts(normalize=True)

N    0.609073
Y    0.390927
Name: smoke, dtype: float64

In [25]:
pd.crosstab(babies["parity"], babies["smoke"], margins=True)

smoke,N,Y,All
parity,,,
N,459,295,754
Y,172,110,282
All,631,405,1036


In [26]:
pd.crosstab(babies["parity"], babies["smoke"], margins=True, normalize=True)

smoke,N,Y,All
parity,,,
N,0.443050,0.284749,0.727799
Y,0.166023,0.106178,0.272201
All,0.609073,0.390927,1.000000


8. Describe the Measures of Central Tendency (mean, median, mode) of the numeric variables in the dataset. Calculate the mean, median, and mode for all 5 numeric variables. For each median and mode, describe the following:

    * What does the median value tell you about the data?
    * What does the mode value tell you about the data?

In [27]:
babies["bwt"].mean()

120.17567567567568

In [28]:
babies["bwt"].mode()

0    117
dtype: int64

In [29]:
babies["bwt"].median() 

120.0

9. Describe the Measures of Variability of the numeric variables in the dataset. Calculate the range, standard deviation, and 85th percentile for all 5 numeric variables. Answer the following questions:

    * Which column has the largest range? Which has the smallest? What does this mean?
    * Which column has the largest standard deviation? Which has the smallest? What does this tell you about those variables?

In [30]:
bwt_range = babies["bwt"].max() - babies["bwt"].min()
print(bwt_range)

90


In [31]:
babies["bwt"].std()

16.40437160501985

In [32]:
np.percentile(babies["bwt"], 85)

137.0